# COMP4471 Project

**Deblurring Rainy Driving Images Using Diffusion Models**  
Lam King Cheuk, Yau Ho Yin, Chan Chun Hugo  



# Mount Drive

In [ ]:
# ============================================================================
# SECTION 1: MOUNT GOOGLE DRIVE
# ============================================================================
# Run this first to access your Google Drive storage

from google.colab import drive
import os

# Mount drive (will skip if already mounted)
try:
    drive.mount("/content/drive")
except:
    print("Drive already mounted or mount failed")

# Navigate to your project folder (create if doesn't exist)
project_dir = "/content/drive/MyDrive/comp4471/project"
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)
print(f"✓ Working directory: {os.getcwd()}")

print(f"Working in: {os.getcwd()}")
!ls -la # Should show your existing notebook/files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Working directory: /content/drive/MyDrive/comp4471/project
Working in: /content/drive/MyDrive/comp4471/project
total 86
drwx------ 8 root root  4096 Dec  3 08:23 data
-rw------- 1 root root 83775 Dec  3 09:22 deRain.ipynb


# Install Dependencies


In [ ]:
# ============================================================================
# SECTION 2: INSTALL DEPENDENCIES
# ============================================================================
# Install only necessary packages for data preparation

print("=" * 60)
print("CHECKING AND INSTALLING DEPENDENCIES")
print("=" * 60)

# List of (pip_package_name, import_name) tuples
packages_to_check = [
    ("tqdm", "tqdm"),
    ("pillow", "PIL"),
    ("opencv-python", "cv2"),
    ("scikit-image", "skimage"),
]

for pip_name, import_name in packages_to_check:
    try:
        __import__(import_name)
        print(f"✓ {pip_name} already installed")
    except ImportError:
        print(f"📥 Installing {pip_name}...")
        get_ipython().system(f"pip install -q {pip_name}")
        print(f"✓ {pip_name} installed")

print("\n✓ All dependencies ready")

CHECKING AND INSTALLING DEPENDENCIES
✓ tqdm already installed
✓ pillow already installed
✓ opencv-python already installed
✓ scikit-image already installed

✓ All dependencies ready


# Dataset Preparation

## 1. Data Unzipping

In [ ]:
# ============================================================================
# SECTION 3: UNZIP BDD100K IMAGES (100K)
# ============================================================================
# Unzip the official BDD100K 100k images with progress tracking

import zipfile
from tqdm import tqdm

print("\n" + "=" * 60)
print("UNZIPPING BDD100K IMAGES (100K)")
print("=" * 60)


def unzip_with_progress(zip_path, extract_to="."):
    """Unzip file with tqdm progress bar"""
    if not os.path.exists(zip_path):
        print(f"❌ File not found: {zip_path}")
        print(f"📁 Available files:")
        get_ipython().system("ls -lh *.zip")
        return False

    print(f"📦 Extracting: {zip_path}")
    print(f"📂 To: {extract_to}")

    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        file_list = zip_ref.namelist()
        total_files = len(file_list)

        print(f"📊 Total files in archive: {total_files:,}")

        for file in tqdm(file_list, desc="Extracting", unit="files"):
            zip_ref.extract(file, extract_to)

    print(f"✓ Extraction complete!")
    return True


# Create data directory
os.chdir("data")

# Unzip images (adjust filename if needed)
# Expected file: bdd100k_images_100k.zip (~36 GB)
images_zip = "bdd100k_images_100k.zip"
images_extract_dir = "100k_images"
os.makedirs(images_extract_dir, exist_ok=True)
if unzip_with_progress(images_zip, images_extract_dir):
    print(f"✓ Images unzipped successfully to {images_extract_dir}/")
else:
    print("⚠️  Please ensure bdd100k_images_100k.zip is in the project root")

print("\n📁 Current structure:")
get_ipython().system("ls -lh")


UNZIPPING BDD100K IMAGES (100K)
📦 Extracting: bdd100k_images_100k.zip
📂 To: 100k_images
📊 Total files in archive: 100,004


Extracting: 100%|██████████| 100004/100004 [29:03<00:00, 57.35files/s]


✓ Extraction complete!
✓ Images unzipped successfully to 100k_images/

📁 Current structure:
total 5.5G
drwx------ 3 root root 4.0K Dec  3 05:34 100k_images
-rw------- 1 root root 5.3G Dec  3 04:52 bdd100k_images_100k.zip
-rw------- 1 root root 181M Dec  3 04:42 bdd100k_labels.zip


In [ ]:
# ============================================================================
# SECTION 4: UNZIP BDD100K LABELS
# ============================================================================
# Unzip the official BDD100K labels

print("\n" + "=" * 60)
print("UNZIPPING BDD100K LABELS")
print("=" * 60)

# Unzip labels (adjust filename if needed)
# Expected file: bdd100k_labels.zip (~2 GB)
labels_zip = "bdd100k_labels.zip"
labels_extract_dir = "100k_labels"
os.makedirs(labels_extract_dir, exist_ok=True)
if unzip_with_progress(labels_zip, labels_extract_dir):
    print(f"✓ Labels unzipped successfully to {labels_extract_dir}/")
else:
    print("⚠️  Please ensure bdd100k_labels.zip is in the project root")

print("\n📁 Final structure:")
get_ipython().system("ls -lhR | head -50")


UNZIPPING BDD100K LABELS
📦 Extracting: bdd100k_labels.zip
📂 To: 100k_labels
📊 Total files in archive: 100,004


Extracting: 100%|██████████| 100004/100004 [29:15<00:00, 56.97files/s]


✓ Extraction complete!
✓ Labels unzipped successfully to 100k_labels/

📁 Final structure:
.:
total 5.5G
drwx------ 3 root root 4.0K Dec  3 05:34 100k_images
drwx------ 3 root root 4.0K Dec  3 06:04 100k_labels
-rw------- 1 root root 5.3G Dec  3 04:52 bdd100k_images_100k.zip
-rw------- 1 root root 181M Dec  3 04:42 bdd100k_labels.zip

./100k_images:
total 4.0K
drwx------ 5 root root 4.0K Dec  3 05:59 100k

./100k_images/100k:
total 12K
drwx------ 2 root root 4.0K Dec  3 06:03 test
drwx------ 2 root root 4.0K Dec  3 05:57 train
drwx------ 2 root root 4.0K Dec  3 05:59 val

./100k_images/100k/test:
total 1.1G
-rw------- 1 root root  50K Dec  3 06:02 cabc30fc-e7726578.jpg
-rw------- 1 root root  56K Dec  3 06:01 cabc30fc-eb673c5a.jpg
-rw------- 1 root root  54K Dec  3 06:02 cabc30fc-fd79926f.jpg
-rw------- 1 root root  38K Dec  3 06:01 cabc9045-1b8282ba.jpg
-rw------- 1 root root  38K Dec  3 06:03 cabc9045-581f64de.jpg
-rw------- 1 root root  41K Dec  3 06:03 cabc9045-5a50690f.jpg
-rw-----

## 2. Filtering

In [ ]:
# ============================================================================
# SECTION 5: ANALYZE DIRECTORY STRUCTURE
# ============================================================================
# Identify the actual directory structure after extraction

print("\n" + "=" * 60)
print("ANALYZING DIRECTORY STRUCTURE")
print("=" * 60)

# Common possible structures after extraction
possible_image_paths = [
    "100k_images/100k/train",  # Actual structure from your extraction
    "100k_images/bdd100k/images/100k/train",
    "100k_images/images/100k/train",
]

# Labels can be either a directory with individual JSON files or a single JSON file
possible_label_paths = [
    "100k_labels/100k/train",  # Directory with individual JSON files (your structure)
    "100k_labels/labels/bdd100k_labels_images_train.json",  # Single JSON file
    "100k_labels/bdd100k_labels_images_train.json",
]

# Find images directory
train_img_dir = None
for path in possible_image_paths:
    if os.path.exists(path):
        train_img_dir = path
        print(f"✓ Found training images at: {train_img_dir}")
        break

if train_img_dir is None:
    # Do a search
    print("🔍 Searching for image directories...")
    get_ipython().system('find . -type d -name "train" | head -10')
    train_img_dir = input("Enter the correct path to train images directory: ")

# Count images
if train_img_dir and os.path.exists(train_img_dir):
    train_images = [
        f
        for f in os.listdir(train_img_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]
    print(f"📊 Found {len(train_images):,} training images")
else:
    print("❌ Training images directory not found!")

# Find labels (can be directory or single file)
train_label_path = None
labels_are_directory = False

for path in possible_label_paths:
    if os.path.exists(path):
        train_label_path = path
        if os.path.isdir(path):
            labels_are_directory = True
            label_files = [f for f in os.listdir(path) if f.endswith('.json')]
            print(f"✓ Found training labels directory: {train_label_path}")
            print(f"  Contains {len(label_files):,} JSON files")
        else:
            print(f"✓ Found training labels file: {train_label_path}")
        break

if train_label_path is None:
    # Do a search
    print("🔍 Searching for label files...")
    get_ipython().system('find . -type f -name "*train*.json" | head -10')
    train_label_path = input("Enter the correct path to train labels: ")
    labels_are_directory = os.path.isdir(train_label_path)

print("\n" + "=" * 60)
print("DETECTED STRUCTURE:")
print("=" * 60)
print(f"Images: {train_img_dir}")
print(f"Labels: {train_label_path} ({'directory' if labels_are_directory else 'single file'})")
print("=" * 60)


ANALYZING DIRECTORY STRUCTURE
✓ Found training images at: 100k_images/100k/train
📊 Found 70,000 training images
✓ Found training labels directory: 100k_labels/100k/train
  Contains 70,000 JSON files

DETECTED STRUCTURE:
Images: 100k_images/100k/train
Labels: 100k_labels/100k/train (directory)


In [ ]:
# ============================================================================
# SECTION 6: LOAD AND PARSE LABELS
# ============================================================================
# Load BDD100K labels and extract weather metadata

import json
from glob import glob

print("\n" + "=" * 60)
print("LOADING TRAINING LABELS")
print("=" * 60)

all_labels = []

if not os.path.exists(train_label_path):
    print(f"❌ ERROR: Label path not found at {train_label_path}")
    print("Please update train_label_path variable with correct path")
else:
    if labels_are_directory:
        # Load individual JSON files
        print(f"📥 Loading individual JSON files from: {train_label_path}")
        json_files = glob(os.path.join(train_label_path, "*.json"))

        for json_file in tqdm(json_files, desc="Loading labels"):
            with open(json_file, "r") as f:
                label_data = json.load(f)
                # Individual files have the structure directly
                all_labels.append(label_data)

        print(f"✓ Loaded {len(all_labels):,} individual label files")
    else:
        # Load single JSON file with all labels
        print(f"📥 Loading labels from single file: {train_label_path}")

        with open(train_label_path, "r") as f:
            all_labels = json.load(f)

        # BDD100K format: list of image metadata
        if isinstance(all_labels, list):
            print(f"✓ Loaded {len(all_labels):,} image labels")
        else:
            print(f"⚠️  Unexpected label format: {type(all_labels)}")

    # Show example
    if len(all_labels) > 0:
        example = all_labels[0]
        print(f"\n📋 Example label structure:")
        print(f"  Name: {example.get('name', 'N/A')}")
        print(f"  Weather: {example.get('attributes', {}).get('weather', 'N/A')}")
        print(f"  Scene: {example.get('attributes', {}).get('scene', 'N/A')}")
        print(f"  Timeofday: {example.get('attributes', {}).get('timeofday', 'N/A')}")


LOADING TRAINING LABELS
📥 Loading individual JSON files from: 100k_labels/100k/train


Loading labels: 100%|██████████| 70000/70000 [07:12<00:00, 161.82it/s]

✓ Loaded 70,000 individual label files

📋 Example label structure:
  Name: 6866acb3-cf17e759
  Weather: clear
  Scene: highway
  Timeofday: daytime


In [ ]:
# ============================================================================
# SECTION 7: BUILD WEATHER MAPPING
# ============================================================================
# Create a dictionary mapping image names to weather conditions

from collections import Counter

print("\n" + "=" * 60)
print("BUILDING WEATHER MAP")
print("=" * 60)

# Build weather map: image_name -> weather_type
weather_map = {}

for item in tqdm(all_labels, desc="Processing labels"):
    img_name = item["name"] + ".jpg"  # BDD100K images are .jpg
    weather = item.get("attributes", {}).get("weather", "clear")
    weather_map[img_name] = weather

print(f"✓ Built weather map for {len(weather_map):,} images")

# Show weather distribution
weather_counts = Counter(weather_map.values())

print("\n☁️  Weather Distribution:")
print("-" * 60)
total = len(weather_map)
for weather, count in weather_counts.most_common():
    percentage = (count / total * 100) if total > 0 else 0
    bar_length = int(percentage / 2)  # Scale to 50 chars max
    bar = "█" * bar_length + "░" * (50 - bar_length)
    print(f"  {weather:15s} │ {bar} │ {count:6,d} ({percentage:5.2f}%)")
print("-" * 60)

# Key statistics
rainy_count = weather_counts.get("rainy", 0)
clear_count = weather_counts.get("clear", 0)
print(f"\n🎯 Target counts:")
print(f"  Rainy images available: {rainy_count:,}")
print(f"  Clear images available: {clear_count:,}")


BUILDING WEATHER MAP


Processing labels: 100%|██████████| 70000/70000 [00:00<00:00, 131469.46it/s]


✓ Built weather map for 70,000 images

☁️  Weather Distribution:
------------------------------------------------------------
  clear           │ ██████████████████████████░░░░░░░░░░░░░░░░░░░░░░░░ │ 37,412 (53.45%)
  overcast        │ ██████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │  8,784 (12.55%)
  undefined       │ █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │  8,134 (11.62%)
  snowy           │ ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │  5,571 ( 7.96%)
  rainy           │ ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │  5,083 ( 7.26%)
  partly cloudy   │ ███░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │  4,886 ( 6.98%)
  foggy           │ ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ │    130 ( 0.19%)
------------------------------------------------------------

🎯 Target counts:
  Rainy images available: 5,083
  Clear images available: 37,412


In [ ]:
# ============================================================================
# SECTION 8: FILTER AND COLLECT RAINY IMAGES
# ============================================================================
# Copy rainy images to organized dataset structure

import shutil

print("\n" + "=" * 60)
print("COLLECTING RAINY IMAGES")
print("=" * 60)

# Create output directories
output_dirs = {
    "train_rainy": "train/rainy",
    "train_clear": "train/clear",
    "val_rainy": "val/rainy",
    "val_clear": "val/clear",
    "test_rainy": "test/rainy",
    "test_clear": "test/clear",
}

for folder in output_dirs.values():
    os.makedirs(folder, exist_ok=True)

# Configuration
TARGET_TRAIN_RAINY = 1200
TARGET_TRAIN_CLEAR = 1200
TARGET_VAL_RAINY = 150
TARGET_VAL_CLEAR = 150
TARGET_TEST_RAINY = 200
TARGET_TEST_CLEAR = 200

# Collect images
print(f"🎯 Collection targets:")
print(f"  Training: {TARGET_TRAIN_RAINY:,} rainy + {TARGET_TRAIN_CLEAR:,} clear")
print(f"  Validation: {TARGET_VAL_RAINY:,} rainy + {TARGET_VAL_CLEAR:,} clear")
print(f"  Test: {TARGET_TEST_RAINY:,} rainy + {TARGET_TEST_CLEAR:,} clear")
print()

# Verify image directory exists
if not os.path.exists(train_img_dir):
    print(f"❌ ERROR: Image directory not found: {train_img_dir}")
else:
    # Get all image files
    all_image_files = [
        f
        for f in os.listdir(train_img_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"📊 Scanning {len(all_image_files):,} images from {train_img_dir}")
    print()

    # Separate images by weather
    rainy_images = []
    clear_images = []

    for img_name in tqdm(all_image_files, desc="Filtering by weather"):
        weather = weather_map.get(img_name, "unknown")

        if weather == "rainy":
            rainy_images.append(img_name)
        elif weather == "clear":
            clear_images.append(img_name)

    print(f"\n✓ Filtered images:")
    print(f"  Rainy: {len(rainy_images):,}")
    print(f"  Clear: {len(clear_images):,}")

    # Shuffle for random distribution
    import random

    random.seed(42)
    random.shuffle(rainy_images)
    random.shuffle(clear_images)

    # Split into train/val/test
    # Rainy split
    rainy_train = rainy_images[:TARGET_TRAIN_RAINY]
    rainy_val = rainy_images[TARGET_TRAIN_RAINY : TARGET_TRAIN_RAINY + TARGET_VAL_RAINY]
    rainy_test = rainy_images[
        TARGET_TRAIN_RAINY + TARGET_VAL_RAINY : TARGET_TRAIN_RAINY
        + TARGET_VAL_RAINY
        + TARGET_TEST_RAINY
    ]

    # Clear split
    clear_train = clear_images[:TARGET_TRAIN_CLEAR]
    clear_val = clear_images[TARGET_TRAIN_CLEAR : TARGET_TRAIN_CLEAR + TARGET_VAL_CLEAR]
    clear_test = clear_images[
        TARGET_TRAIN_CLEAR + TARGET_VAL_CLEAR : TARGET_TRAIN_CLEAR
        + TARGET_VAL_CLEAR
        + TARGET_TEST_CLEAR
    ]

    print(f"\n📦 Split distribution:")
    print(f"  Train: {len(rainy_train):,} rainy + {len(clear_train):,} clear")
    print(f"  Val:   {len(rainy_val):,} rainy + {len(clear_val):,} clear")
    print(f"  Test:  {len(rainy_test):,} rainy + {len(clear_test):,} clear")



COLLECTING RAINY IMAGES
🎯 Collection targets:
  Training: 1,200 rainy + 1,200 clear
  Validation: 150 rainy + 150 clear
  Test: 200 rainy + 200 clear

📊 Scanning 70,000 images from 100k_images/100k/train



Filtering by weather: 100%|██████████| 70000/70000 [00:00<00:00, 424326.52it/s]


✓ Filtered images:
  Rainy: 5,083
  Clear: 37,412



📦 Split distribution:
  Train: 1,200 rainy + 1,200 clear
  Val:   150 rainy + 150 clear
  Test:  200 rainy + 200 clear


## 3. Saving

In [ ]:
# ============================================================================
# SECTION 9: COPY IMAGES TO ORGANIZED STRUCTURE
# ============================================================================
# Copy filtered images to train/val/test directories

print("\n" + "=" * 60)
print("COPYING IMAGES TO ORGANIZED STRUCTURE")
print("=" * 60)


def copy_images(image_list, src_dir, dest_dir, desc):
    """Copy images with progress bar using subprocess (faster on Google Drive)"""
    import subprocess

    copied = 0
    for img_name in tqdm(image_list, desc=desc, unit="imgs"):
        src_path = os.path.join(src_dir, img_name)
        dest_path = os.path.join(dest_dir, img_name)

        if os.path.exists(src_path):
            try:
                # Use subprocess cp command (much faster on mounted drives)
                subprocess.run(['cp', src_path, dest_path], check=True, capture_output=True)
                copied += 1
            except subprocess.CalledProcessError:
                # Fallback to shutil if cp fails
                shutil.copy(src_path, dest_path)
                copied += 1

    return copied


# Copy all splits
print("📋 Copying training set...")
train_rainy_copied = copy_images(
    rainy_train, train_img_dir, "train/rainy", "🌧️  Train rainy"
)
train_clear_copied = copy_images(
    clear_train, train_img_dir, "train/clear", "☀️  Train clear"
)

print("\n📋 Copying validation set...")
val_rainy_copied = copy_images(rainy_val, train_img_dir, "val/rainy", "🌧️  Val rainy")
val_clear_copied = copy_images(clear_val, train_img_dir, "val/clear", "☀️  Val clear")

print("\n📋 Copying test set...")
test_rainy_copied = copy_images(
    rainy_test, train_img_dir, "test/rainy", "🌧️  Test rainy"
)
test_clear_copied = copy_images(
    clear_test, train_img_dir, "test/clear", "☀️  Test clear"
)

print("\n" + "=" * 60)
print("COPY SUMMARY")
print("=" * 60)
print(
    f"Train: {train_rainy_copied:,} rainy + {train_clear_copied:,} clear = {train_rainy_copied + train_clear_copied:,}"
)
print(
    f"Val:   {val_rainy_copied:,} rainy + {val_clear_copied:,} clear = {val_rainy_copied + val_clear_copied:,}"
)
print(
    f"Test:  {test_rainy_copied:,} rainy + {test_clear_copied:,} clear = {test_rainy_copied + test_clear_copied:,}"
)
print(
    f"TOTAL: {train_rainy_copied + val_rainy_copied + test_rainy_copied + train_clear_copied + val_clear_copied + test_clear_copied:,} images"
)
print("=" * 60)


COPYING IMAGES TO ORGANIZED STRUCTURE
📋 Copying training set...


☀️  Train clear: 100%|██████████| 1200/1200 [00:22<00:00, 52.85imgs/s]



📋 Copying validation set...


☀️  Val clear: 100%|██████████| 150/150 [00:03<00:00, 48.30imgs/s]



📋 Copying test set...


☀️  Test clear: 100%|██████████| 200/200 [00:04<00:00, 41.85imgs/s]


COPY SUMMARY
Train: 1,200 rainy + 1,200 clear = 2,400
Val:   150 rainy + 150 clear = 300
Test:  200 rainy + 200 clear = 400
TOTAL: 3,100 images


## 4. Compress extracted files

In [ ]:
# ============================================================================
# SECTION 10: CREATE SHAREABLE DATASET ARCHIVE
# ============================================================================
# Package organized dataset for teammates (simple zip file)

import zipfile

print("\n" + "=" * 60)
print("CREATING SHAREABLE ARCHIVE FOR TEAMMATES")
print("=" * 60)

# Create compressed zip archive
archive_name = "derrain_dataset.zip"
print(f"📦 Creating archive: {archive_name}")
print("   This may take a few minutes...")

try:
    with zipfile.ZipFile(archive_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        print("   Adding train/ folder...")
        for root, dirs, files in os.walk("train/"):
            for file in tqdm(files, desc="   Compressing train", leave=False):
                file_path = os.path.join(root, file)
                arcname = os.path.join("data", file_path)
                zipf.write(file_path, arcname)

        print("   Adding val/ folder...")
        for root, dirs, files in os.walk("val/"):
            for file in tqdm(files, desc="   Compressing val", leave=False):
                file_path = os.path.join(root, file)
                arcname = os.path.join("data", file_path)
                zipf.write(file_path, arcname)

        print("   Adding test/ folder...")
        for root, dirs, files in os.walk("test/"):
            for file in tqdm(files, desc="   Compressing test", leave=False):
                file_path = os.path.join(root, file)
                arcname = os.path.join("data", file_path)
                zipf.write(file_path, arcname)

        # Add metadata if it exists (created in Section 11)
        metadata_file = "dataset_info.json"
        if os.path.exists(metadata_file):
            print("   Adding metadata...")
            zipf.write(metadata_file, "data/dataset_info.json")
        else:
            print(f"   ⚠️  Skipping metadata (not found at: {os.path.abspath(metadata_file)})")
            print(f"   ℹ️  Metadata will be created in Section 11")

    # Calculate archive size
    archive_size_bytes = os.path.getsize(archive_name)
    archive_size_mb = archive_size_bytes / (1024 * 1024)
    archive_size_gb = archive_size_bytes / (1024 * 1024 * 1024)

    print(f"\n✓ Archive created successfully!")
    print("=" * 60)
    print(f"📦 Archive Details:")
    print(f"   File: {archive_name}")
    if archive_size_gb >= 1.0:
        print(f"   Size: {archive_size_gb:.2f} GB ({archive_size_mb:.0f} MB)")
    else:
        print(f"   Size: {archive_size_mb:.0f} MB")
    print(f"   Location: {os.path.join(os.getcwd(), archive_name)}")
    print("=" * 60)

    print(f"\n📤 Share this file with your teammates!")
    print(f"\n🔧 Extraction command for teammates:")
    print(f"   unzip {archive_name}")
    print(f"\n📋 Contents:")
    print(f"   • data/train/rainy/ (1,200 images)")
    print(f"   • data/train/clear/ (1,200 images)")
    print(f"   • data/val/rainy/ (150 images)")
    print(f"   • data/val/clear/ (150 images)")
    print(f"   • data/test/rainy/ (200 images)")
    print(f"   • data/test/clear/ (200 images)")
    if os.path.exists("dataset_info.json"):
        print(f"   • data/dataset_info.json (metadata)")

except Exception as e:
    print(f"\n❌ Error creating archive: {e}")
    print("You can create it manually with:")
    print(f"   zip -r {archive_name} train/ val/ test/ dataset_info.json")


CREATING SHAREABLE ARCHIVE FOR TEAMMATES
📦 Creating archive: derrain_dataset.zip
   This may take a few minutes...
   Adding train/ folder...


   Adding val/ folder...


   Adding test/ folder...


   ⚠️  Skipping metadata (not found at: /content/drive/MyDrive/comp4471/project/data/dataset_info.json)
   ℹ️  Metadata will be created in Section 11

✓ Archive created successfully!
📦 Archive Details:
   File: derrain_dataset.zip
   Size: 156 MB
   Location: /content/drive/MyDrive/comp4471/project/data/derrain_dataset.zip

📤 Share this file with your teammates!

🔧 Extraction command for teammates:
   unzip derrain_dataset.zip

📋 Contents:
   • data/train/rainy/ (1,200 images)
   • data/train/clear/ (1,200 images)
   • data/val/rainy/ (150 images)
   • data/val/clear/ (150 images)
   • data/test/rainy/ (200 images)
   • data/test/clear/ (200 images)


In [ ]:
# ============================================================================
# SECTION 11: SAVE DATASET METADATA
# ============================================================================
# Create metadata file with dataset statistics

from datetime import datetime

print("\n" + "=" * 60)
print("SAVING DATASET METADATA")
print("=" * 60)

metadata = {
    "dataset_name": "BDD100K DriveDeRain Dataset",
    "creation_date": datetime.now().isoformat(),
    "source": "BDD100K: Images 100K (Official)",
    "image_resolution": "1280x720 (original)",
    "splits": {
        "train": {
            "rainy": train_rainy_copied,
            "clear": train_clear_copied,
            "total": train_rainy_copied + train_clear_copied,
        },
        "val": {
            "rainy": val_rainy_copied,
            "clear": val_clear_copied,
            "total": val_rainy_copied + val_clear_copied,
        },
        "test": {
            "rainy": test_rainy_copied,
            "clear": test_clear_copied,
            "total": test_rainy_copied + test_clear_copied,
        },
    },
    "totals": {
        "rainy": train_rainy_copied + val_rainy_copied + test_rainy_copied,
        "clear": train_clear_copied + val_clear_copied + test_clear_copied,
        "total_images": train_rainy_copied
        + val_rainy_copied
        + test_rainy_copied
        + train_clear_copied
        + val_clear_copied
        + test_clear_copied,
    },
    "paths": {
        "source_images": train_img_dir,
        "source_labels": train_label_path,
        "output_root": os.getcwd(),
    },
}

# Save metadata
metadata_file = "dataset_info.json"
with open(metadata_file, "w") as f:
    json.dump(metadata, f, indent=2)

print(f"✓ Metadata saved to: {metadata_file}")

# Display metadata
print("\n" + "=" * 60)
print("DATASET METADATA")
print("=" * 60)
print(json.dumps(metadata, indent=2))


SAVING DATASET METADATA
✓ Metadata saved to: dataset_info.json

DATASET METADATA
{
  "dataset_name": "BDD100K DriveDeRain Dataset",
  "creation_date": "2025-12-03T08:18:59.068185",
  "source": "BDD100K: Images 100K (Official)",
  "image_resolution": "1280x720 (original)",
  "splits": {
    "train": {
      "rainy": 1200,
      "clear": 1200,
      "total": 2400
    },
    "val": {
      "rainy": 150,
      "clear": 150,
      "total": 300
    },
    "test": {
      "rainy": 200,
      "clear": 200,
      "total": 400
    }
  },
  "totals": {
    "rainy": 1550,
    "clear": 1550,
    "total_images": 3100
  },
  "paths": {
    "source_images": "100k_images/100k/train",
    "source_labels": "100k_labels/100k/train",
    "output_root": "/content/drive/MyDrive/comp4471/project/data"
  }
}


In [ ]:
# ============================================================================
# SECTION 12: VERIFY DATASET INTEGRITY
# ============================================================================
# Verify all images are readable and correctly organized

from PIL import Image

print("\n" + "=" * 60)
print("VERIFYING DATASET INTEGRITY")
print("=" * 60)


def verify_images(directory):
    """Verify all images in directory are readable"""
    issues = []
    image_files = [
        f
        for f in os.listdir(directory)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    for img_file in tqdm(
        image_files, desc=f"Verifying {os.path.basename(directory)}", unit="imgs"
    ):
        img_path = os.path.join(directory, img_file)
        try:
            img = Image.open(img_path)
            img.verify()  # Verify it's a valid image
        except Exception as e:
            issues.append(f"{img_file}: {str(e)}")

    return len(image_files), issues


# Verify all splits
total_verified = 0
total_issues = []

for split in ["train", "val", "test"]:
    for condition in ["rainy", "clear"]:
        directory = f"{split}/{condition}"
        if os.path.exists(directory):
            count, issues = verify_images(directory)
            total_verified += count
            total_issues.extend(issues)

print(f"\n✓ Verified {total_verified:,} images")

if total_issues:
    print(f"⚠️  Found {len(total_issues)} issues:")
    for issue in total_issues[:10]:  # Show first 10
        print(f"  - {issue}")
    if len(total_issues) > 10:
        print(f"  ... and {len(total_issues) - 10} more")
else:
    print("✓ All images are valid and readable!")


VERIFYING DATASET INTEGRITY


Verifying clear: 100%|██████████| 200/200 [00:00<00:00, 297.59imgs/s]


✓ Verified 3,100 images
✓ All images are valid and readable!


In [ ]:
# ============================================================================
# SECTION 13: DISPLAY FINAL SUMMARY
# ============================================================================
# Show final dataset organization and next steps

print("\n" + "=" * 80)
print(" " * 25 + "DATA PREPARATION COMPLETE!")
print("=" * 80)

print("\n📊 DATASET SUMMARY:")
print(f"  Total images: {metadata['totals']['total_images']:,}")
print(f"  Rainy images: {metadata['totals']['rainy']:,}")
print(f"  Clear images: {metadata['totals']['clear']:,}")

print("\n📁 DIRECTORY STRUCTURE:")
print(f"  {os.getcwd()}/")
print(f"    ├── train/")
print(f"    │   ├── rainy/     ({train_rainy_copied:,} images)")
print(f"    │   └── clear/     ({train_clear_copied:,} images)")
print(f"    ├── val/")
print(f"    │   ├── rainy/     ({val_rainy_copied:,} images)")
print(f"    │   └── clear/     ({val_clear_copied:,} images)")
print(f"    ├── test/")
print(f"    │   ├── rainy/     ({test_rainy_copied:,} images)")
print(f"    │   └── clear/     ({test_clear_copied:,} images)")
print(f"    └── dataset_info.json")

print("\n📋 FILES SAVED:")
print(f"  Dataset root: {os.getcwd()}")
print(f"  Metadata: {os.path.join(os.getcwd(), metadata_file)}")

print("\n🎯 NEXT STEPS:")
print("  1. ✓ Dataset preparation complete")
print("  2. → Generate synthetic rain (optional)")
print("  3. → Train derain model (Pix2Pix / Diffusion)")
print("  4. → Run evaluation (PSNR / SSIM / YOLO mAP)")

print("\n💡 USAGE:")
print("  - Training pairs: use train/rainy as input, train/clear as target")
print("  - Validation: use val/rainy and val/clear for monitoring")
print("  - Testing: use test/rainy and test/clear for final evaluation")

print("\n" + "=" * 80)
print("Ready to proceed with model training! 🚀")
print("=" * 80)


                         DATA PREPARATION COMPLETE!

📊 DATASET SUMMARY:
  Total images: 3,100
  Rainy images: 1,550
  Clear images: 1,550

📁 DIRECTORY STRUCTURE:
  /content/drive/MyDrive/comp4471/project/data/
    ├── train/
    │   ├── rainy/     (1,200 images)
    │   └── clear/     (1,200 images)
    ├── val/
    │   ├── rainy/     (150 images)
    │   └── clear/     (150 images)
    ├── test/
    │   ├── rainy/     (200 images)
    │   └── clear/     (200 images)
    └── dataset_info.json

📋 FILES SAVED:
  Dataset root: /content/drive/MyDrive/comp4471/project/data
  Metadata: /content/drive/MyDrive/comp4471/project/data/dataset_info.json

🎯 NEXT STEPS:
  1. ✓ Dataset preparation complete
  2. → Generate synthetic rain (optional)
  3. → Train derain model (Pix2Pix / Diffusion)
  4. → Run evaluation (PSNR / SSIM / YOLO mAP)

💡 USAGE:
  - Training pairs: use train/rainy as input, train/clear as target
  - Validation: use val/rainy and val/clear for monitoring
  - Testing: use test/ra

# Sync with Google Drive

In [34]:
# Add to end of prepare.py
from google.colab import drive
import time
# Force sync
drive.flush_and_unmount()
time.sleep(5)
drive.mount('/content/drive')

Mounted at /content/drive
